In [1]:
import tkinter as tk
from tkinter import filedialog
from tkinter import messagebox
import arcpy
import pandas
import os
import glob

<div style="height: 30px;"></div>

### Step 1: Select the File Path for Main Folder

In [2]:
# Open a folder selection dialog
folder_path = filedialog.askdirectory(title="Select a Folder")
confirm = messagebox.askyesno("Confirm Folder", f"You selected:\n\n{folder_path}\n\nIs this correct?")

<div style="height: 30px;"></div>

### Step 2: Check for Existing Geodatabases and Create if Necessary

In [3]:
# Specify the folder path where you want to create the geodatabase
cst_gdb_name = 'Coastal.gdb'
cst_output_workspace = os.path.join(folder_path, cst_gdb_name)

riv_gdb_name = 'Riverine.gdb'
riv_output_workspace = os.path.join(folder_path, riv_gdb_name)


# Check if the Coastal geodatabase already exists
if not arcpy.Exists(cst_output_workspace):
    # Create the geodatabase
    arcpy.management.CreateFileGDB(folder_path, 'Coastal')
    print(f"Geodatabase created at: {cst_output_workspace}")
else:
    print(f"Geodatabase already exists at: {cst_output_workspace}")
    
    
    
# Check if the Riverine geodatabase already exists
if not arcpy.Exists(riv_output_workspace):
    # Create the geodatabase
    arcpy.management.CreateFileGDB(folder_path, 'Riverine')
    print(f"Geodatabase created at: {riv_output_workspace}")
else:
    print(f"Geodatabase already exists at: {riv_output_workspace}")

Geodatabase created at: C:/git/FEMA TOOL\Coastal.gdb
Geodatabase created at: C:/git/FEMA TOOL\Riverine.gdb


<div style="height: 30px;"></div>

<div style="height: 30px;"></div>

### Step 4: Process Coastal Feature Classes and Merge



In [4]:
# Set Coastal Workspace
arcpy.env.workspace = cst_output_workspace

#Create Coastal Combined Feature Class Name
cst_combined_name = "Coastal_Combined_S_AOI_Ar_WGS84"
geometry_type = "POLYGON"

# Create the feature class
arcpy.CreateFeatureclass_management(cst_output_workspace, cst_combined_name, geometry_type)

# Set the spatial reference to WGS84
spatial_reference = arcpy.SpatialReference(4326)  # WGS84 WKID
arcpy.DefineProjection_management(cst_combined_name, spatial_reference)


# List of fields to add (same as before)
field_list = [
    {"name": "AOI_ID", "type": "TEXT", "length": 25},
    {"name": "FIPS", "type": "TEXT", "length": 5},
    {"name": "POL_NAME1", "type": "TEXT", "length": 50},
    {"name": "AOI_TYP", "type": "TEXT", "length": 25},
    {"name": "AOI_ISSUE", "type": "TEXT", "length": 100},
    {"name": "AOI_INFO", "type": "TEXT", "length": 254},
    {"name": "NOTES", "type": "TEXT", "length": 254},
]


# Add fields to the feature class
for field in field_list:
    arcpy.AddField_management(cst_combined_name, field["name"], field["type"], field_length=field.get("length"))

<div style="height: 30px;"></div>

### Step 5: Process Riverine Feature Classes and Merge

In [5]:
# Set Coastal Workspace
arcpy.env.workspace = riv_output_workspace

#Create Coastal Combined Feature Class Name
riv_combined_name = "Riverine_Combined_S_AOI_Ar_WGS84"
geometry_type = "POLYGON" 

# Create the feature class
arcpy.CreateFeatureclass_management(riv_output_workspace, riv_combined_name, geometry_type)

# Set the spatial reference to WGS84
spatial_reference = arcpy.SpatialReference(4326)  # WGS84 WKID
arcpy.DefineProjection_management(riv_combined_name, spatial_reference)


# List of fields to add (same as before)
field_list = [
    {"name": "AOI_ID", "type": "TEXT", "length": 25},
    {"name": "FIPS", "type": "TEXT", "length": 5},
    {"name": "POL_NAME1", "type": "TEXT", "length": 50},
    {"name": "AOI_TYP", "type": "TEXT", "length": 25},
    {"name": "AOI_ISSUE", "type": "TEXT", "length": 100},
    {"name": "AOI_INFO", "type": "TEXT", "length": 254},
    {"name": "NOTES", "type": "TEXT", "length": 254},
]


# Add fields to the feature class
for field in field_list:
    arcpy.AddField_management(riv_combined_name, field["name"], field["type"], field_length=field.get("length"))

<div style="height: 30px;"></div>

### Step 6: Compare and Confirm

In [6]:
# Compare the features
compare_result = arcpy.management.FeatureCompare(
    in_base_features=cst_combined_name,
    in_test_features=riv_combined_name,
    sort_field = "OBJECTID",
    continue_compare="CONTINUE_COMPARE",
    out_compare_file=None  # No need to save a report file
)


# Check if there are any differences (ignore geometry differences)
if compare_result.getOutput(1) == "true":
    print("The schemas match.")
else:
    print("The schemas do not match.")

The schemas match.
